In [68]:
!pip install pandas
!pip install gender-guesser

In [69]:
import pandas as pd
import numpy as np
import gender_guesser.detector as gender

#### Load Datasets

In [70]:
baby_names = pd.read_csv("../data/Popular_Baby_Names_20240424.csv")
expense_budget = pd.read_csv("../data/Expense_Budget_20240424.csv", low_memory=False)
payroll = pd.read_csv("../data/Citywide_Payroll_Data__Fiscal_Year__20240424.csv", low_memory=False)

#### Getting Gender Information

Checking overlap between baby names and payroll data

In [71]:
unique_baby = baby_names[baby_names["Child's First Name"].notnull()]
unique_payroll = payroll[payroll["First Name"].notnull()]

unique_baby = baby_names["Child's First Name"].unique()
unique_payroll = payroll["First Name"].unique()

unique_payroll = unique_payroll.astype(str)
unique_baby = unique_baby.astype(str)

unique_baby = np.char.upper(unique_baby)
unique_payroll = np.char.upper(unique_payroll)

len(np.intersect1d(unique_baby, unique_payroll))

1804

Use a gender prediction model

In [72]:
genModel = gender.Detector()

In [73]:
print(genModel.get_gender("Pauley"))

andy


#### Data Cleaning

In [74]:
payroll.isnull().any(axis=0)

Fiscal Year                   False
Payroll Number                 True
Agency Name                   False
Last Name                      True
First Name                     True
Mid Init                       True
Agency Start Date              True
Work Location Borough          True
Title Description              True
Leave Status as of June 30    False
Base Salary                   False
Pay Basis                     False
Regular Hours                 False
Regular Gross Paid            False
OT Hours                      False
Total OT Paid                 False
Total Other Pay               False
dtype: bool

In [79]:
payroll[payroll["Work Location Borough"].notna()

SyntaxError: incomplete input (1734138353.py, line 1)

Filter out all the rows that are missing names. We can't do gender analysis on them. There are 13,245 rows with missing names. 

In [76]:
payroll = payroll[payroll["First Name"].notna()]

Filter out all the rows that are missing start dates. There are only 63 rows with missing start dates. Turn the agency start date into a numeric time value. This will make doing time calculations a lot easier.

In [77]:
payroll = payroll[payroll["Agency Start Date"].notna()]

#turn all dates into linux time


There are 506,233 rows with missing Work Location Borough. Since the number of missing names is large, we shouldn't just get rid of them. Some people might work in many different buroughs due to the nature of their jobs. Since this is a categorical value, we can have an extra value "Unknown" for any one with missing burough information. 

In [78]:
payroll = payroll["Work Location Borough"].fillna("UNKNOWN")